## Churn Prediction

required table :
1. customer
2. cust_order
3. order_status
4. order_history
5. address
6. address_status
7. order_line
8. shipping_method

In [283]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

path = r"D:\DS_Final_Project\Notebook\data\RAW_CSV"

customer = pd.read_csv(f"{path}\\customer.csv")
cust_order = pd.read_csv(f"{path}\\cust_order.csv")
customer_address = pd.read_csv(f"{path}\\customer_address.csv")
address = pd.read_csv(f"{path}\\address.csv")
address_status = pd.read_csv(f"{path}\\address_status.csv")
order_history = pd.read_csv(f"{path}\\order_history.csv")
order_line = pd.read_csv(f"{path}\\order_line.csv")
order_status = pd.read_csv(f"{path}\\order_status.csv")
shipping_method = pd.read_csv(f"{path}\\shipping_method.csv")


In [18]:
tables = [
    ("customer", customer),
    ("cust_order", cust_order),
    ("customer_address", customer_address),
    ("address", address),
    ("address_status", address_status),
    ("order_history", order_history),
    ("order_line", order_line),
    ("order_status", order_status),
    ("shipping_method", shipping_method)]

for name,df in tables:
    print(f"{name} : {list(df.columns)}")
    print("-"*50)


customer : ['customer_id', 'first_name', 'last_name', 'email']
--------------------------------------------------
cust_order : ['order_id', 'order_date', 'customer_id', 'shipping_method_id', 'dest_address_id']
--------------------------------------------------
customer_address : ['customer_id', 'address_id', 'status_id']
--------------------------------------------------
address : ['address_id', 'street_number', 'street_name', 'city', 'country_id']
--------------------------------------------------
address_status : ['status_id', 'address_status']
--------------------------------------------------
order_history : ['history_id', 'order_id', 'status_id', 'status_date']
--------------------------------------------------
order_line : ['line_id', 'order_id', 'book_id', 'price']
--------------------------------------------------
order_status : ['status_id', 'status_value']
--------------------------------------------------
shipping_method : ['method_id', 'method_name', 'cost']
---------------

In [ ]:
#customer & cust_order & customer_address == customer id
#cust_order & order_history & order line == order_id
#cust_order & customer_address & address == address_id
#customer_address & address_status == status_id
#order_history & order_status == status id  (Both are different)

In [233]:
customer.sample(10)

,customer_id,first_name,last_name,email
890,891,Alvis,Atkirk,aatkirkoq@vk.com
1251,1252,Etti,Staziker,estaziker6z@biblegateway.com
490,491,Jayme,Benge,jbengedm@goodreads.com
1346,1347,Omar,Woolcocks,owoolcocks9m@tamu.edu
996,997,Phaedra,Moresby,pmoresbyro@desdev.cn
291,292,Redd,Foyston,rfoyston83@zimbio.com
1508,1509,Tann,Booy,tbooye4@odnoklassniki.ru
182,183,Urbanus,Renfield,urenfield52@aboutads.info
166,167,Hervey,Erik,herik4m@zimbio.com
1129,1130,Marline,Neiland,mneiland3l@sphinn.com


In [234]:
#checking for any null values
customer.isnull().sum()

customer_id    0
first_name     0
last_name      0
email          0
dtype: int64

In [3]:
#checking for duplicates
print(customer.duplicated().sum())

0


In [117]:
# Check for the shape
customer['customer_id'].shape

(2000,)

In [284]:
# Taking copy of the dataframe to retain the original df
customer_df = customer.copy()
customer_df

,customer_id,first_name,last_name,email
0,1,Ursola,Purdy,upurdy0@cdbaby.com
1,2,Ruthanne,Vatini,rvatini1@fema.gov
2,3,Reidar,Turbitt,rturbitt2@geocities.jp
3,4,Rich,Kirsz,rkirsz3@jalbum.net
4,5,Carline,Kupis,ckupis4@tamu.edu
...,...,...,...,...
1995,1996,Byron,Mayor,bmayorrn@netscape.com
1996,1997,Janean,Coatsworth,jcoatsworthro@purevolume.com
1997,1998,Georgeanna,Garman,ggarmanrp@surveymonkey.com
1998,1999,Ardeen,Caret,acaretrq@wsj.com


In [236]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  2000 non-null   int64 
 1   first_name   2000 non-null   object
 2   last_name    2000 non-null   object
 3   email        2000 non-null   object
dtypes: int64(1), object(3)
memory usage: 62.6+ KB


In [285]:
# Taking copy of the dataframe to retain the original df

cust_order_df = cust_order.copy()

In [219]:
cust_order_df.shape

(7550, 5)

In [286]:
cust_order_df.isnull().sum()

order_id              0
order_date            0
customer_id           0
shipping_method_id    0
dest_address_id       0
dtype: int64

In [220]:
print(cust_order_df.duplicated().sum())

0


In [20]:
cust_order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7550 entries, 0 to 7549
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            7550 non-null   int64 
 1   order_date          7550 non-null   object
 2   customer_id         7550 non-null   int64 
 3   shipping_method_id  7550 non-null   int64 
 4   dest_address_id     7550 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 295.1+ KB


In [221]:
cust_order_df.head()

,order_id,order_date,customer_id,shipping_method_id,dest_address_id
0,1,2023-08-22 16:12:12,413,2,1
1,2,2023-07-25 03:52:27,1338,1,1
2,3,2022-05-26 14:15:06,1464,3,2
3,4,2022-01-01 10:23:19,269,2,3
4,5,2024-11-29 11:35:26,1463,4,3


In [287]:
pd.set_option('display.max_column',50)

In [288]:
cust_order_df["order_date"] = pd.to_datetime(cust_order_df["order_date"])
cust_order_df['cust_order_day'] = cust_order_df['order_date'].dt.day
cust_order_df['cust_order_month'] = cust_order_df['order_date'].dt.month
cust_order_df['cust_order_year'] = cust_order_df["order_date"].dt.year
cust_order_df['order_date_formatted'] = cust_order_df["order_date"].dt.strftime('%m-%d-%Y')
cust_order_df.drop(["order_date"],axis=1,inplace=True)
cust_order_df

,order_id,customer_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted
0,1,413,2,1,22,8,2023,08-22-2023
1,2,1338,1,1,25,7,2023,07-25-2023
2,3,1464,3,2,26,5,2022,05-26-2022
3,4,269,2,3,1,1,2022,01-01-2022
4,5,1463,4,3,29,11,2024,11-29-2024
...,...,...,...,...,...,...,...,...
7545,11430,1850,4,13,26,12,2023,12-26-2023
7546,11431,673,1,14,2,4,2022,04-02-2022
7547,11432,427,2,15,22,5,2024,05-22-2024
7548,11433,506,4,15,22,8,2023,08-22-2023


In [289]:
cust_order_df.isnull().sum()

order_id                0
customer_id             0
shipping_method_id      0
dest_address_id         0
cust_order_day          0
cust_order_month        0
cust_order_year         0
order_date_formatted    0
dtype: int64

In [290]:
# Taking copy of Order line table
order_line_df = order_line.copy()


In [291]:
order_line_df.sample(5)

,line_id,order_id,book_id,price
3964,3965,8666,8824,7.61
9264,9906,10865,4611,7.74
7052,7053,132,6492,9.01
1164,1165,1047,1838,18.22
13759,14543,5013,7743,8.04


In [292]:
order_line_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15400 entries, 0 to 15399
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   line_id   15400 non-null  int64  
 1   order_id  15400 non-null  int64  
 2   book_id   15400 non-null  int64  
 3   price     15400 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 481.4 KB


In [129]:
order_line_df.isnull().sum()

line_id     0
order_id    0
book_id     0
price       0
dtype: int64

In [163]:
print(order_line_df.duplicated().sum())

0


In [245]:
order_line_df

,line_id,order_id,book_id,price
0,1,1213,6476,13.42
1,2,5308,6784,0.70
2,3,240,3857,12.55
3,4,2333,1091,12.14
4,5,4335,8258,17.79
...,...,...,...,...
15395,16424,11428,6825,0.91
15396,16425,8315,1606,13.99
15397,16426,4695,8707,17.82
15398,16427,6550,10090,10.23


In [293]:
#Merging customer table and cust_order table

customer_cust_order_df = pd.merge(customer,cust_order_df,how="left",on="customer_id")
customer_cust_order_df.sample(5)

,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted
1368,328,Aguistin,Trounce,atrounce93@hibu.com,6984.0,2.0,254.0,1.0,3.0,2022.0,03-01-2022
3863,955,Mercedes,Gorger,mgorgerqi@bizjournals.com,4815.0,4.0,219.0,9.0,5.0,2022.0,05-09-2022
4648,1186,Elfreda,Gadd,egadd55@imdb.com,6157.0,2.0,6.0,28.0,12.0,2023.0,12-28-2023
1155,272,Alaster,Lemmers,alemmers7j@cornell.edu,8648.0,1.0,144.0,12.0,4.0,2022.0,04-12-2022
1499,365,Lucy,Jacklin,ljacklina4@earthlink.net,5375.0,3.0,379.0,29.0,4.0,2022.0,04-29-2022


In [294]:
# Merging customer_cust_order and  order line df

merged_customer_df = pd.merge(customer_cust_order_df,order_line_df,how='left',on='order_id')
merged_customer_df.sample(5)

,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted,line_id,book_id,price
1786,217,Rustin,Cadden,rcadden60@goo.ne.jp,9516.0,3.0,93.0,19.0,6.0,2024.0,06-19-2024,10145.0,8920.0,4.63
3058,367,Charlton,Brimner,cbrimnera6@digg.com,5331.0,2.0,369.0,22.0,6.0,2024.0,06-22-2024,9001.0,10562.0,8.46
6397,805,Agnes,Dunnico,adunnicomc@independent.co.uk,8698.0,1.0,159.0,8.0,8.0,2023.0,08-08-2023,3062.0,9335.0,0.86
6903,867,Patin,Casaletto,pcasalettoo2@globo.com,383.0,2.0,118.0,14.0,6.0,2023.0,06-14-2023,9361.0,45.0,11.07
2683,317,Stesha,McAlees,smcalees8s@wikispaces.com,10629.0,4.0,122.0,19.0,10.0,2024.0,10-19-2024,16171.0,3553.0,9.57


In [295]:
merged_customer_df["order_date_formatted"] = pd.to_datetime(merged_customer_df["order_date_formatted"])

merged_customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15699 entries, 0 to 15698
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           15699 non-null  int64         
 1   first_name            15699 non-null  object        
 2   last_name             15699 non-null  object        
 3   email                 15699 non-null  object        
 4   order_id              15400 non-null  float64       
 5   shipping_method_id    15400 non-null  float64       
 6   dest_address_id       15400 non-null  float64       
 7   cust_order_day        15400 non-null  float64       
 8   cust_order_month      15400 non-null  float64       
 9   cust_order_year       15400 non-null  float64       
 10  order_date_formatted  15400 non-null  datetime64[ns]
 11  line_id               15400 non-null  float64       
 12  book_id               15400 non-null  float64       
 13  price           

In [296]:
merged_customer_df.isnull().sum()

customer_id               0
first_name                0
last_name                 0
email                     0
order_id                299
shipping_method_id      299
dest_address_id         299
cust_order_day          299
cust_order_month        299
cust_order_year         299
order_date_formatted    299
line_id                 299
book_id                 299
price                   299
dtype: int64

In [297]:
#notna() returns true or false when we have Non-null(not for missing values)
merged_customer_df['has_order'] = merged_customer_df['order_id'].notna().astype(int)
merged_customer_df.sample(5)

,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted,line_id,book_id,price,has_order
7576,936,Elmore,Peyto,epeytopz@github.io,9559.0,1.0,109.0,6.0,4.0,2024.0,2024-04-06,3628.0,6265.0,18.88,1
10251,1308,Dion,Nys,dnys8j@ucoz.com,7103.0,3.0,282.0,7.0,12.0,2023.0,2023-12-07,4922.0,5388.0,16.80,1
1842,227,Greta,Wharby,gwharby6a@nyu.edu,5314.0,2.0,363.0,26.0,6.0,2024.0,2024-06-26,877.0,2633.0,13.48,1
615,83,Berti,Gowlett,bgowlett2a@pagesperso-orange.fr,800.0,3.0,241.0,20.0,1.0,2022.0,2022-01-20,293.0,8450.0,14.52,1
7354,914,Ivar,Corona,icoronapd@joomla.org,9256.0,4.0,13.0,9.0,6.0,2022.0,2022-06-09,10285.0,9603.0,9.66,1


In [298]:
merged_customer_df[merged_customer_df['order_id'].isnull()].sample(5)

,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted,line_id,book_id,price,has_order
13437,1726,Kordula,Ronisch,kronischk5@yandex.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0
13015,1674,Dolf,Lasty,dlastyip@columbia.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0
14258,1828,Melissa,Giraudel,mgiraudelmz@ezinearticles.com,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0
10078,1287,Yanaton,Tuny,ytuny7y@google.ca,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0
3733,473,Simona,Prahm,sprahmd4@joomla.org,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,0


In [253]:
merged_customer_df.isnull().sum()

customer_id               0
first_name                0
last_name                 0
email                     0
order_id                299
shipping_method_id      299
dest_address_id         299
cust_order_day          299
cust_order_month        299
cust_order_year         299
order_date_formatted    299
line_id                 299
book_id                 299
price                   299
has_order                 0
dtype: int64

In [299]:
# Fill missing order-related columns with a 0 value
merged_customer_df['order_id'] = merged_customer_df['order_id'].fillna(0).astype(int)
merged_customer_df['shipping_method_id'] = merged_customer_df['shipping_method_id'].fillna(0).astype(int)
merged_customer_df['dest_address_id'] = merged_customer_df['dest_address_id'].fillna(0).astype(int)
merged_customer_df['cust_order_day'] = merged_customer_df['cust_order_day'].fillna(0).astype(int)
merged_customer_df['cust_order_month'] = merged_customer_df['cust_order_month'].fillna(0).astype(int)
merged_customer_df['cust_order_year'] = merged_customer_df['cust_order_year'].fillna(0).astype(int)

merged_customer_df['order_date_formatted'] = merged_customer_df['order_date_formatted'].fillna('No Date')

merged_customer_df['line_id'] = merged_customer_df['line_id'].fillna(0).astype(int)
merged_customer_df['book_id'] = merged_customer_df['book_id'].fillna(0).astype(int)
merged_customer_df['price'] = merged_customer_df['price'].fillna(0).astype(float)  


In [300]:
merged_customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15699 entries, 0 to 15698
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customer_id           15699 non-null  int64  
 1   first_name            15699 non-null  object 
 2   last_name             15699 non-null  object 
 3   email                 15699 non-null  object 
 4   order_id              15699 non-null  int64  
 5   shipping_method_id    15699 non-null  int64  
 6   dest_address_id       15699 non-null  int64  
 7   cust_order_day        15699 non-null  int64  
 8   cust_order_month      15699 non-null  int64  
 9   cust_order_year       15699 non-null  int64  
 10  order_date_formatted  15699 non-null  object 
 11  line_id               15699 non-null  int64  
 12  book_id               15699 non-null  int64  
 13  price                 15699 non-null  float64
 14  has_order             15699 non-null  int64  
dtypes: float64(1), int6

In [301]:
merged_customer_df.isnull().sum()

customer_id             0
first_name              0
last_name               0
email                   0
order_id                0
shipping_method_id      0
dest_address_id         0
cust_order_day          0
cust_order_month        0
cust_order_year         0
order_date_formatted    0
line_id                 0
book_id                 0
price                   0
has_order               0
dtype: int64

In [302]:
order_history_df = order_history.copy()

In [261]:
order_history_df.duplicated().sum()

np.int64(0)

In [262]:
order_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22349 entries, 0 to 22348
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   history_id   22349 non-null  int64 
 1   order_id     22349 non-null  int64 
 2   status_id    22349 non-null  int64 
 3   status_date  22349 non-null  object
dtypes: int64(3), object(1)
memory usage: 698.5+ KB


In [303]:
order_history_df.isnull().sum()

history_id     0
order_id       0
status_id      0
status_date    0
dtype: int64

In [174]:
order_history_df.columns

Index(['history_id', 'order_id', 'status_id', 'status_date'], dtype='object')

In [304]:
#Formatting the status date

order_history_df['status_date'] = pd.to_datetime(order_history_df['status_date'])
order_history_df['order_hist_day'] = order_history_df['status_date'].dt.day
order_history_df['order_hist_month_num'] = order_history_df['status_date'].dt.month
order_history_df['order_hist_month_name'] = order_history_df['status_date'].dt.month_name()
order_history_df['order_hist_year'] = order_history_df['status_date'].dt.year
order_history_df['hist_status_date'] = order_history_df['status_date'].dt.strftime('%m-%d-%Y')
order_history_df['hist_status_date'] = pd.to_datetime(order_history_df['hist_status_date'])
order_history_df.drop("status_date",inplace=True,axis=1)
order_history_df.sample(5)

,history_id,order_id,status_id,order_hist_day,order_hist_month_num,order_hist_month_name,order_hist_year,hist_status_date
237,238,238,1,22,9,September,2023,2023-09-22
15585,17619,8525,3,27,3,March,2024,2024-03-27
7981,8624,5138,2,10,12,December,2023,2023-12-10
17076,19110,1345,3,5,8,August,2023,2023-08-05
5733,5734,8724,1,29,2,February,2024,2024-02-29


In [305]:
order_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22349 entries, 0 to 22348
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   history_id             22349 non-null  int64         
 1   order_id               22349 non-null  int64         
 2   status_id              22349 non-null  int64         
 3   order_hist_day         22349 non-null  int32         
 4   order_hist_month_num   22349 non-null  int32         
 5   order_hist_month_name  22349 non-null  object        
 6   order_hist_year        22349 non-null  int32         
 7   hist_status_date       22349 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(3), int64(3), object(1)
memory usage: 1.1+ MB


In [306]:
# Taking a copy of order line table
order_status_df = order_status.copy()
order_status_df

,status_id,status_value
0,1,Order Received
1,2,Pending Delivery
2,3,Delivery In Progress
3,4,Delivered
4,5,Cancelled
5,6,Returned


In [307]:
order_status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_id     6 non-null      int64 
 1   status_value  6 non-null      object
dtypes: int64(1), object(1)
memory usage: 228.0+ bytes


In [308]:
print(order_history_df.shape, order_status.shape)

(22349, 8) (6, 2)


In [310]:
# Merging order history df and order status df.

hist_status_df = pd.merge(order_history_df,order_status_df,how='left',on='status_id')
hist_status_df.head()


,history_id,order_id,status_id,order_hist_day,order_hist_month_num,order_hist_month_name,order_hist_year,hist_status_date,status_value
0,1,1,1,22,8,August,2023,2023-08-22,Order Received
1,2,2,1,25,7,July,2023,2023-07-25,Order Received
2,3,3,1,26,5,May,2022,2022-05-26,Order Received
3,4,4,1,1,1,January,2022,2022-01-01,Order Received
4,5,5,1,29,11,November,2024,2024-11-29,Order Received


In [271]:
merged_customer_df.columns

Index(['customer_id', 'first_name', 'last_name', 'email', 'order_id',
       'shipping_method_id', 'dest_address_id', 'cust_order_day',
       'cust_order_month', 'cust_order_year', 'order_date_formatted',
       'line_id', 'book_id', 'price', 'has_order'],
      dtype='object')

In [182]:
hist_status_df.columns

Index(['history_id', 'order_id', 'status_id', 'order_hist_day',
       'order_hist_month_num', 'order_hist_month_name', 'order_hist_year',
       'hist_status_date', 'status_value'],
      dtype='object')

In [311]:
merged_customer_df = pd.merge(merged_customer_df,hist_status_df,how='left',on='order_id')
merged_customer_df.sample(5)

,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted,line_id,book_id,price,has_order,history_id,status_id,order_hist_day,order_hist_month_num,order_hist_month_name,order_hist_year,hist_status_date,status_value
1638,68,Renado,Sherrington,rsherrington1v@symantec.com,6780,1,197,10,8,2022,2022-08-10 00:00:00,255,8032,19.92,1,4637.0,1.0,10.0,8.0,August,2022.0,2022-08-10,Order Received
287,11,Teriann,Marritt,tmarritta@va.gov,5330,3,369,29,7,2023,2023-07-29 00:00:00,8503,7010,0.49,1,11933.0,2.0,30.0,7.0,July,2023.0,2023-07-30,Pending Delivery
4591,191,Esmeralda,Swinfen,eswinfen5a@ovh.net,4232,2,41,6,8,2024,2024-08-06 00:00:00,8845,10598,10.07,1,17894.0,3.0,8.0,8.0,August,2024.0,2024-08-08,Delivery In Progress
27128,1180,Allx,Juara,ajuara4z@yahoo.co.jp,2330,3,707,27,6,2022,2022-06-27 00:00:00,4458,4529,6.15,1,2329.0,1.0,27.0,6.0,June,2022.0,2022-06-27,Order Received
16905,733,Aguste,Alleburton,aalleburtonkc@yellowbook.com,5074,1,287,22,12,2021,2021-12-22 00:00:00,2760,8437,17.92,1,9869.0,2.0,24.0,12.0,December,2021.0,2021-12-24,Pending Delivery


In [312]:
# Convert the 'order_date_formatted' to a different format

merged_customer_df['order_date_formatted'] = pd.to_datetime(merged_customer_df['order_date_formatted'],errors='coerce')
merged_customer_df['order_date_formatted'] = merged_customer_df['order_date_formatted'].dt.strftime('%Y-%m-%d')

# Print the result
merged_customer_df


,customer_id,first_name,last_name,email,order_id,shipping_method_id,dest_address_id,cust_order_day,cust_order_month,cust_order_year,order_date_formatted,line_id,book_id,price,has_order,history_id,status_id,order_hist_day,order_hist_month_num,order_hist_month_name,order_hist_year,hist_status_date,status_value
0,1,Ursola,Purdy,upurdy0@cdbaby.com,1213,4,359,7,2,2023,2023-02-07,1,6476,13.42,1,1212.0,1.0,7.0,2.0,February,2023.0,2023-02-07,Order Received
1,1,Ursola,Purdy,upurdy0@cdbaby.com,1213,4,359,7,2,2023,2023-02-07,1,6476,13.42,1,12860.0,2.0,8.0,2.0,February,2023.0,2023-02-08,Pending Delivery
2,1,Ursola,Purdy,upurdy0@cdbaby.com,1213,4,359,7,2,2023,2023-02-07,1,6476,13.42,1,18936.0,3.0,8.0,2.0,February,2023.0,2023-02-08,Delivery In Progress
3,1,Ursola,Purdy,upurdy0@cdbaby.com,1213,4,359,7,2,2023,2023-02-07,1,6476,13.42,1,21072.0,4.0,9.0,2.0,February,2023.0,2023-02-09,Delivered
4,1,Ursola,Purdy,upurdy0@cdbaby.com,1213,4,359,7,2,2023,2023-02-07,1,6476,13.42,1,25124.0,6.0,11.0,2.0,February,2023.0,2023-02-11,Returned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45910,2000,Delora,Bigglestone,dbigglestonerr@usatoday.com,9532,4,99,28,5,2022,2022-05-28,9939,3557,18.04,1,19475.0,3.0,30.0,5.0,May,2022.0,2022-05-30,Delivery In Progress
45911,2000,Delora,Bigglestone,dbigglestonerr@usatoday.com,9532,4,99,28,5,2022,2022-05-28,9939,3557,18.04,1,23532.0,4.0,1.0,6.0,June,2022.0,2022-06-01,Delivered
45912,2000,Delora,Bigglestone,dbigglestonerr@usatoday.com,10555,4,99,22,5,2024,2024-05-22,7550,1577,19.27,1,7019.0,1.0,22.0,5.0,May,2024.0,2024-05-22,Order Received
45913,2000,Delora,Bigglestone,dbigglestonerr@usatoday.com,10555,4,99,22,5,2024,2024-05-22,7550,1577,19.27,1,14753.0,2.0,23.0,5.0,May,2024.0,2024-05-23,Pending Delivery


In [313]:
merged_customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45915 entries, 0 to 45914
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customer_id            45915 non-null  int64         
 1   first_name             45915 non-null  object        
 2   last_name              45915 non-null  object        
 3   email                  45915 non-null  object        
 4   order_id               45915 non-null  int64         
 5   shipping_method_id     45915 non-null  int64         
 6   dest_address_id        45915 non-null  int64         
 7   cust_order_day         45915 non-null  int64         
 8   cust_order_month       45915 non-null  int64         
 9   cust_order_year        45915 non-null  int64         
 10  order_date_formatted   45616 non-null  object        
 11  line_id                45915 non-null  int64         
 12  book_id                45915 non-null  int64         
 13  p

In [314]:
merged_customer_df.isnull().sum()

customer_id                0
first_name                 0
last_name                  0
email                      0
order_id                   0
shipping_method_id         0
dest_address_id            0
cust_order_day             0
cust_order_month           0
cust_order_year            0
order_date_formatted     299
line_id                    0
book_id                    0
price                      0
has_order                  0
history_id               300
status_id                300
order_hist_day           300
order_hist_month_num     300
order_hist_month_name    300
order_hist_year          300
hist_status_date         300
status_value             300
dtype: int64

In [280]:
# Create a flag to indicate whether a customer has order history
merged_customer_df['has_order_history'] = merged_customer_df['history_id'].notna().astype(int)


In [281]:
# Correct way to handle missing values without inplace=True
merged_customer_df['history_id'] = merged_customer_df['history_id'].fillna(0)
merged_customer_df['status_id'] = merged_customer_df['status_id'].fillna(0)
merged_customer_df['order_hist_day'] = merged_customer_df['order_hist_day'].fillna(0)
merged_customer_df['order_hist_month_num'] = merged_customer_df['order_hist_month_num'].fillna(0)
merged_customer_df['order_hist_month_name'] = merged_customer_df['order_hist_month_name'].fillna('No History')
merged_customer_df['order_hist_year'] = merged_customer_df['order_hist_year'].fillna(0)
merged_customer_df['hist_status_date'] = merged_customer_df['hist_status_date'].fillna('No History')
merged_customer_df['status_value'] = merged_customer_df['status_value'].fillna('No History')

# For order_date_formatted column
#merged_customer_df['order_date_formatted'] = merged_customer_df['order_date_formatted'].fillna('No Date')


In [282]:
pd.set_option('display.max_columns',50)